# Factor risk models

In [1]:
import pandas as pd
import numpy as np
import cvxpy as cvx

from cvx.risk.linalg import pca
from cvx.risk.factor import FactorModel
from cvx.portfolio.min_risk import minrisk_problem

## Load prices and compute returns

In [2]:
prices = pd.read_csv("data/stock_prices.csv", index_col=0, header=0, parse_dates=True)
returns = prices.pct_change().fillna(0.0)

## Compute principal components

In [3]:
factors = pca(returns=returns, n_components=10)

## Create the risk model, here a FactorModel

In [4]:
model = FactorModel(assets=len(returns.columns), k=10)

# update the model parameters
model.update(cov=factors.cov, exposure=factors.exposure.values, idiosyncratic_risk=factors.idiosyncratic.std().values,
             lower_assets=np.zeros(20), upper_assets=np.ones(20),
             lower_factors=-0.1*np.ones(10), upper_factors=0.1*np.ones(10))

# test the risk model with uniform weights
weights = 0.05 * np.ones(20)
model.estimate(weights).value

np.float64(0.009234077305378861)

## RiskModel is injected into optimizer

In [5]:
w = cvx.Variable(20)
y = cvx.Variable(10)

problem = minrisk_problem(model, w, y=y)
problem.solve()

print(pd.Series(data=w.value, index=prices.columns))
print(model.estimate(w, y=y).value)

# check the solution
assert np.isclose(w.value.sum(), 1.0)
assert np.all(w.value > -0.01)
print(y.value)

GOOG    5.654445e-09
AAPL    1.078298e-02
FB      5.488994e-09
BABA    6.149758e-02
AMZN    4.095639e-02
GE      1.694607e-03
AMD     2.165087e-09
WMT     4.448256e-02
BAC     2.516558e-09
GM      2.730308e-09
T       1.297914e-01
UAA     2.310979e-09
SHLD    1.733013e-09
XOM     2.502611e-01
RRC     2.457944e-09
BBY     1.392807e-02
MA      5.914495e-02
PFE     2.364385e-01
JPM     8.938543e-09
SBUX    1.510218e-01
dtype: float64
0.006799927163237368
[ 0.04251151  0.09313457  0.03753817  0.08220493 -0.08363883 -0.0280072
 -0.1        -0.07172425  0.09783884  0.03309744]


In [6]:
y.value

array([ 0.04251151,  0.09313457,  0.03753817,  0.08220493, -0.08363883,
       -0.0280072 , -0.1       , -0.07172425,  0.09783884,  0.03309744])

In [7]:
model.parameter["exposure"].value @ w.value

array([ 0.04251151,  0.09313457,  0.03753817,  0.08220493, -0.08363883,
       -0.0280072 , -0.1       , -0.07172425,  0.09783884,  0.03309744])

In [8]:
y.value

array([ 0.04251151,  0.09313457,  0.03753817,  0.08220493, -0.08363883,
       -0.0280072 , -0.1       , -0.07172425,  0.09783884,  0.03309744])

In [9]:
model.parameter["exposure"].value @ w.value

array([ 0.04251151,  0.09313457,  0.03753817,  0.08220493, -0.08363883,
       -0.0280072 , -0.1       , -0.07172425,  0.09783884,  0.03309744])

In [10]:
y.value

array([ 0.04251151,  0.09313457,  0.03753817,  0.08220493, -0.08363883,
       -0.0280072 , -0.1       , -0.07172425,  0.09783884,  0.03309744])

In [11]:
model.parameter["exposure"].value @ w.value

array([ 0.04251151,  0.09313457,  0.03753817,  0.08220493, -0.08363883,
       -0.0280072 , -0.1       , -0.07172425,  0.09783884,  0.03309744])